In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import inception_base
import os
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.client import device_lib

from model import Caption_Model
from data_generator import Data_Generator
from inference_util import Inference

import inception_base
import configuration
import argparse

In [2]:
from tensorflow.python.layers.core import Dense

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", help="Batch size of train data input",
                    type=int,default=64)
parser.add_argument("num_epochs", help="Number of epochs to train the model.",
                    type=int)
parser.add_argument("--checkpoint_model", help="Model Checkpoint to use.",
                    type=str, default=None)
parser.add_argument("--summary_freq", help="Frequency of writing summary to tensorboard.",
                    type=int, default=100)
parser.add_argument("--save_freq", help="Frequency of saving model.",
                    type=int, default=None)

_StoreAction(option_strings=['--save_freq'], dest='save_freq', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help='Frequency of saving model.', metavar=None)

In [4]:
FLAGS = parser.parse_args(["20","--save_freq","500"])

In [5]:
data_config = configuration.DataConfig().config
data_gen = Data_Generator(processed_video_dir = data_config["processed_video_dir"],
                         caption_file = data_config["caption_file"],
                         unique_freq_cutoff = data_config["unique_frequency_cutoff"],
                         max_caption_len = data_config["max_caption_length"])

data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])

In [6]:
model_config = configuration.ModelConfig(data_gen).config

In [7]:
model = Caption_Model( **model_config,mode="train")

In [8]:
model.build()

In [9]:
summary_op = tf.summary.merge(model.summaries)
sess = tf.Session()

In [10]:
train_writer = tf.summary.FileWriter(data_config["train_log_dir"],sess.graph)
saver = tf.train.Saver(max_to_keep=200,keep_checkpoint_every_n_hours=0.5)
sess.run(tf.global_variables_initializer())

In [11]:
data_gen.init_batch(int(FLAGS.batch_size),"train")

In [12]:
iter_to_save = np.int32(FLAGS.save_freq)

In [13]:
epoch=0
i=0

In [39]:
start_time = time.time()
dataset = data_gen.get_next_batch("train")
data_gen_time = time.time() - start_time

feed_dict={}
feed_dict[model.video_mask] = np.ones([dataset["video"].shape[0],dataset["video"].shape[1]],dtype=np.int32)
feed_dict[model.caption_input] = dataset["indexed_caption"]
feed_dict[model.caption_mask] = dataset["caption_mask"]
feed_dict[model.rnn_input] = dataset["video"]
feed_dict[model.is_training] = True

In [40]:
length,mask,model_output,seq_len,model_mask,model_truth = sess.run([model.max_decoder_length,model.mask,model.train_output_rnn,model.train_output_len,model.caption_mask,model.caption_input],feed_dict=feed_dict)

In [41]:
print("Length",length)

Length 19


In [42]:
np.sum(mask,axis=1)

array([  7.,   8.,  11.,  10.,  16.,  13.,  13.,   6.,  11.,   9.,   9.,
         7.,   8.,   8.,  15.,   7.,  10.,   8.,   8.,   8.,   7.,  14.,
        17.,  14.,  19.,   9.,  10.,  16.,  13.,   8.,   8.,  12.,  10.,
         9.,  11.,  13.,   8.,   7.,   7.,   9.,   7.,   8.,  12.,   8.,
         8.,  15.,  15.,  15.,  14.,  12.,  10.,   6.,   8.,   5.,  12.,
         7.,   5.,  10.,   7.,  11.,   7.,   8.,  10.,  13.], dtype=float32)

In [ ]:
loss = sess.run(model.batch_loss,feed_dict=feed_dict)

In [ ]:
np.max(np.sum(model_mask,axis=1))

In [ ]:
model_output.shape

In [ ]:
(model_mask[:,1:]).shape

In [ ]:
model_truth.shape

In [ ]:
model_truth[:,1:].shape

In [ ]:
seq_len

In [ ]:
tf.sequence_mask()

In [ ]:
tf.contrib.seq2seq.sequence_loss()

In [1]:
import tensorflow as tf

In [ ]:
tf.contrib.seq2seq.BeamSearchDecoder()